In [ ]:
import numpy as np
import sys, os
import subprocess
import matplotlib.pyplot as plt
from shutil import copyfile
# adjust some settings for matplotlib
from matplotlib import rcParams
import pynoddy
# print rcParams
# determine path of repository to set paths corretly below
#repo_path = os.path.realpath('../..')

In [3]:
basepynoddyfile = pynoddy.__file__[:-11]+'experiment/__init__.py'
# Read in the file
with open(basepynoddyfile, 'r') as file :
  filedata = file.read()

# Replace the target string
filedata = filedata.replace('from . import util.sampling as Sample', 'from .util import sampling as Sample')

# Write the file out again
with open(basepynoddyfile, 'w') as file:
  file.write(filedata)
  
target = pynoddy.__file__[:-11]+'output.py'

source = 'output.py'

copyfile(source, target)

NameError: name 'pynoddy' is not defined

In [4]:
import pynoddy.history
import pynoddy.experiment
import pynoddy.events

In [5]:
folder = !pwd
folder = folder[0]
noddyEXE = folder+'/noddy.exe'
strV = 'chmod 777 '+noddyEXE
!eval "$strV"

In [6]:
modelfile = 'ReallyNiceTrentonModel.his'
#pynoddyEXEfile = '/content/DataAndCode/noddy_Mac.exe'
output_name = 'noddy_out'

H1 = pynoddy.history.NoddyHistory(modelfile)
H1.change_cube_size(250)
H1.events[3].properties['Dip'] = 40
H1.events[4].properties['Dip'] = 40
H1.events[5].properties['Dip'] = 40
H1.events[5].properties['X'] = 8000
H1.write_history(modelfile)
print('cube size is ' +str(H1.get_cube_size()))
print(subprocess.Popen([noddyEXE, modelfile, output_name, 'ALL'], 
                       shell=False, stderr=subprocess.PIPE, 
                       stdout=subprocess.PIPE).stdout.read())

cube size is 250.0
b''


In [7]:
!ls

apt.txt			      noddy_Mac      noddy_out.vtr
auto_rendering.ipynb	      noddy_out.dxf  output.py
core.python.23.1578085425     noddy_out.g00  Points.csv
DisplacementTransferZone.his  noddy_out.g01  postBuild
DTZ_3d.png		      noddy_out.g02  ReallyNiceTrentonModel.his
environment.yml		      noddy_out.g12  setup.py
noddyBatchProgress.txt	      noddy_out.grv  WellNamesPaths.csv
noddy.exe		      noddy_out.mag


In [8]:
import ezdxf
import pandas as pd
filename = 'noddy_out.dxf'
doc = ezdxf.readfile(filename)

def print_entity(e):
    print("LINE on layer: %s\n" % e.dxf.layer)
    print("start point1: %s\n" % e.dxf.vtx0)
    print("end point2: %s\n" % e.dxf.vtx1)
    print("end point3: %s\n" % e.dxf.vtx2)
    print("end point4: %s\n" % e.dxf.vtx3)


msp = doc.modelspace()
faces = msp.query('3DFACE')
num3Dfaces=len(faces)
print(num3Dfaces)

# %%


i=0
x = np.zeros((num3Dfaces*3, 1))
y = np.zeros((num3Dfaces*3, 1))
z = np.zeros((num3Dfaces*3, 1))
name = []
for e in faces:
    x[i] = e.dxf.vtx0[0]
    y[i] = e.dxf.vtx0[1]
    z[i] = e.dxf.vtx0[2]
    name.append(e.dxf.layer)

    x[i+1] = e.dxf.vtx1[0]
    y[i+1] = e.dxf.vtx1[1]
    z[i+1] = e.dxf.vtx1[2]
    name.append(e.dxf.layer)

    x[i+2] = e.dxf.vtx2[0]
    y[i+2] = e.dxf.vtx2[1]
    z[i+2] = e.dxf.vtx2[2]
    name.append(e.dxf.layer)
    i=i+3
print(i)

# %%
#remove row duplicates        
AllPoints = pd.DataFrame({'x': x.reshape((-1,)), 'y':y.reshape((-1,)), 'z': z.reshape((-1,)), 'lith': name})
AllPoints = AllPoints.drop_duplicates(subset=['x', 'y', 'z'], keep='first')
AllPoints.to_csv('Points.csv', index=False)


170025
510075


In [9]:
output_name = 'noddy_out'
N1 = pynoddy.output.NoddyOutput(output_name)

In [10]:
print("The model has an extent of %.0f m in x-direction, with %d cells of width %.0f m" %
      (N1.extent_x, N1.nx, N1.delx))

The model has an extent of 14000 m in x-direction, with 56 cells of width 250 m


In [11]:
N1.export_to_vtk()

In [9]:
import pandas as pd
from vtkplotter import *
import numpy as np
import matplotlib.pylab as pl

def GetCat(strCat):
    CatCodes = np.zeros((len(strCat),))
    filterB = (strCat['lith'].str.contains('B')) 
    filterS = (strCat['lith'].str.contains('S')) 
    
    CatCodes[filterB]= strCat.loc[filterB, 'lith'].str[:-16].astype('category').cat.codes
    CatCodes[filterS]= -1*strCat.loc[filterS, 'lith'].str[:-12].astype('category').cat.codes

#    CatCodes[filterB]= 0
#    CatCodes[filterS]= 1

    return CatCodes


embedWindow('k3d') #uncomment to pop the usual vtk rendering window, press Esc to close it.

positions = np.random.randn(5000,3)

data = pd.read_csv('Points.csv')
data['Cat'] = GetCat(data)

positions2 = data[['x', 'y','z']].values
scalars = data['Cat']

plot = Plotter()
plot.xtitle = '\sum_i x_i'


listSurfaces = np.unique(data['Cat'])
nSurfaces = len(listSurfaces)
colors = pl.cm.jet(np.linspace(0,1,nSurfaces))

for i in range(nSurfaces):
    surfaceId = listSurfaces[i]
    filterSurface = data['Cat']==surfaceId
    points = Points(positions2[filterSurface]).color(colors[i][0:3])
    # color vertices based on their scalar value with any matplotlib color map
#    points.pointColor(colors[i])
#    plot += points
    points = smoothMLS2D(points, f=0.4)  # smooth cloud, input actor is modified

    print("Nr of points before cleaning polydata:", pts1.N())

    # impose a min distance among mesh points
    points.clean(tol=0.01).legend("smooth cloud")
    print("             after  cleaning polydata:", pts1.N())

    reco = recoSurface(points, bins=128).legend("surf reco")
    plot += reco

#filename = 'noddy_out.vtr'
#e = load(filename, threshold=50)
#value = e.info.keys()
#scals = e.scalars('geology')
#e.pointColors(scals, cmap='Set3')

#plot += e

plot.show()

Recalculating normals for 4931 Points, sample size= 98
Calculating mesh from points with R = 542.9252451182724
Recalculating normals for 4858 Points, sample size= 97
Calculating mesh from points with R = 542.9057500487331
Recalculating normals for 4814 Points, sample size= 96
Calculating mesh from points with R = 542.8936039204636
Recalculating normals for 4826 Points, sample size= 96
Calculating mesh from points with R = 542.8735178764671
Recalculating normals for 4025 Points, sample size= 80
Calculating mesh from points with R = 542.7531031516357
Recalculating normals for 4552 Points, sample size= 91
Calculating mesh from points with R = 542.7467424395546
Recalculating normals for 4615 Points, sample size= 92
Calculating mesh from points with R = 542.8352232599074
Recalculating normals for 4435 Points, sample size= 88
Calculating mesh from points with R = 542.8148496533743
Recalculating normals for 4446 Points, sample size= 88
Calculating mesh from points with R = 542.8053952966469
R

Plot(antialias=3, axes=['\\sum_i x_i', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[6161.490…

In [2]:
np.unique(data['Cat'])

array([-15., -14., -13., -12., -11., -10.,  -9.,  -8.,  -7.,  -6.,  -5.,
        -4.,  -3.,  -2.,  -1.,   0.,   1.,   2.,   3.,   4.,   5.,   6.])

In [7]:
colors[i]

array([0. , 0. , 0.5, 1. ])